In [2]:
import json
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import JSONLoader
from langchain.storage import InMemoryStore
from langchain_core.documents import Document

import os
from dotenv import load_dotenv

In [3]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.storage import InMemoryStore
from langchain_core.documents import Document

import os
from dotenv import load_dotenv

# Load environment variables (make sure to set GOOGLE_API_KEY in your .env file)
load_dotenv()

def read_file_with_fallback_encoding(file_path):
    encodings = ['utf-8', 'latin-1', 'ascii', 'utf-16', 'cp1252']
    for encoding in encodings:
        try:
            with open(file_path, 'r', encoding=encoding) as file:
                return file.read()
        except UnicodeDecodeError:
            continue
    raise ValueError(f"Unable to read the file {file_path} with any of the attempted encodings")

def load_rag_content_from_text(text_file_path, chunk_size=1000, chunk_overlap=200):
    try:
        # Read the file content
        file_content = read_file_with_fallback_encoding(text_file_path)
        
        # Create a Document object
        document = Document(page_content=file_content, metadata={"source": text_file_path})

        # Initialize the text splitter
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
        )

        # Split the document
        splits = text_splitter.split_documents([document])

        # Initialize the embedding function with Gemini Pro
        embedding_function = GoogleGenerativeAIEmbeddings(model="models/embedding-001", task_type='retrieval_query')

        # Create the vector store
        vectorstore = Chroma.from_documents(documents=splits, embedding=embedding_function)

        # Create the retriever
        retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

        return retriever
    except Exception as e:
        print(f"Error processing file {text_file_path}: {str(e)}")
        return None

# Usage
text_file_path = "rag_string.txt"
retriever = load_rag_content_from_text(text_file_path)

In [4]:
if retriever:
    # Test the retriever
    query = "What is the main topic of the content?"
    results = retriever.get_relevant_documents(query)

    for doc in results:
        print(f"Content: {doc.page_content}")
        print(f"Metadata: {doc.metadata}")
        print("-" * 50)
else:
    print("Failed to create retriever due to file reading error.")

c:\Users\Zahid imran\Desktop\LLM\Langchai\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Content: Education.  Home of   Study Abroad  in  Bahrain. Start Your Study Abroad Journey with Us.  We listen carefully to what you want, making your journey smooth. You’ll end up in your dream university.  STUDY ABROAD. JNS Education provides Free counselling service and student visa assistance for students who wants to study abroad. Apply Now For University Admission!.  Official   Testing Center for IELTS PTE, GMAT and more.. We offer a range of convenient test dates for PTE UKVI, GMAT and IELTS UKVI at our test center in Bahrain.  Student Reviews. JnS Education is the one and only Pearson test of English (PTE) center in Bahrain. They also give consultancy for higher education in abroad and specific to the UK. JnS Education works closely with university and candidate.  I recently used JNS for help with my university and visa application for US. The consultant was extremely helpful and knowledgeable throughout the entire process, answering all of my questions and providing guidance wh

In [5]:
import os
from dotenv import load_dotenv
from langchain import PromptTemplate, LLMChain
from langchain.chains import RetrievalQA
from langchain_google_genai import GoogleGenerativeAI

# Load environment variables (make sure to set GOOGLE_API_KEY in your .env file)
load_dotenv()

# Initialize the LLM (assuming you have the Gemini model set up)
llm = GoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.7) 

# Template to format the prompt for Gemini, incorporating the retrieved context
template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Answer:"""
prompt_template = PromptTemplate(input_variables=["context", "question"], template=template)

# Chain to combine the retriever and the LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever, 
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt_template}
)

# Function to get user input and generate a response
def get_chatbot_response(query):
    # Get relevant context from the retriever
    result = qa_chain({"query": query})

    # Return the answer and the source documents used for generating the answer
    return result['result'], result['source_documents'] 

# Chatbot loop
while True:
    query = input("Enter your query (or 'exit' to quit): ")
    if query.lower() == 'exit':
        break

    response, source_documents = get_chatbot_response(query)
    print("Answer:", response)

    # # Optionally, print the source documents used for generating the response
    # if source_documents:
    #     print("\nSources:")
    #     for doc in source_documents:
    #         print(doc.metadata['source'])

c:\Users\Zahid imran\Desktop\LLM\Langchai\venv\Lib\site-packages\langchain_core\_api\deprecation.py:141: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  warn_deprecated(


Answer: JnS Education is the one and only Pearson test of English (PTE) center in Bahrain. They also give consultancy for higher education in abroad and specific to the UK. 

Answer: The provided text does not list all countries where JnS Education provides services. It only mentions that they provide services in 7 major countries worldwide. 

Answer: PTE stands for Pearson Test of English. 

JnS Education is an official testing center for PTE in Bahrain, Pakistan, Kuwait, Saudi Arabia, UAE, and Qatar. They offer a range of convenient test dates for PTE UKVI, PTE-A, and GMAT at their test centers. 

Answer: JnS Education is an official test center for PTE UKVI, PTE-A, and GMAT in Pakistan, Kuwait, Saudi Arabia, UAE, Bahrain, and Qatar. 

Answer: JnS Education is a leading education consultant and PTE test center in Pakistan. They offer:

* **Consultancy services:** Expert consultants help students pursue higher education abroad.
* **Test center:** They are an official test center for P